# Result Analysis of the Car Network

## General Configuration and Functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

## Reserved Bandwidth

In [ ]:
# read omnet results into data frame
reservedBandwidth = pd.read_csv('results/reservedBandwidth.csv')
filtered_data = reservedBandwidth[reservedBandwidth['name'] == 'reservedBandwidth:vector']
filtered_data = filtered_data.dropna(subset=['vecvalue'])
modulePortBandwidth = dict()
for row in filtered_data.iterrows():
    max_value = max(list(map(float, row[1]['vecvalue'].split())))
    if max_value <= 0:
        continue
    module_split = row[1]['module'].split('.')
    node = module_split[1]
    port = module_split[2].split('[')[1].replace(']', '')
    pcp = module_split[-1].split('[')[1].replace(']', '')
    if node not in modulePortBandwidth:
        modulePortBandwidth[module_split[1]] = dict()
    if port not in modulePortBandwidth[node]:
        modulePortBandwidth[node][port] = dict()
    modulePortBandwidth[node][port][pcp] = max_value
    if "total" not in modulePortBandwidth[node][port]:
        modulePortBandwidth[node][port]["total"] = max_value
    else:
        modulePortBandwidth[node][port]["total"] += max_value
print (modulePortBandwidth)

In [ ]:
# plot the modulePortBandwidth for each module, port and pcp combination in a bar chart
fig, ax = plt.subplots()
bar_width = 0.35
opacity = 0.8
for node in modulePortBandwidth:
    for port in modulePortBandwidth[node]:
        for pcp in modulePortBandwidth[node][port]:
            name = node + "_port" + port + "_pcp" + pcp
            ax.bar(name, modulePortBandwidth[node][port][pcp], bar_width, alpha=opacity, label=name)
ax.set_xlabel('Module')
ax.set_ylabel('Bandwidth')
plt.xticks(rotation=90)
# increase space between x-axis labels
ax.set_title('Bandwidth per module, port and pcp')
plt.show()


In [ ]:
# print modulePortBandwidth where total bandwidth is greater than 750000000
for node in modulePortBandwidth:
    for port in modulePortBandwidth[node]:
        if modulePortBandwidth[node][port]["total"] > 750000000:
            print(node, port, modulePortBandwidth[node][port]["total"])

# Analyse flow Updates and Final Config

In [2]:
def warnOverReservation(switchSlopes):
    switches = dict()
    for data in switchSlopes:
        switch = data['switch']
        port = data['port']
        slope = data['idleSlope']
        if switch not in switches:
            switches[switch] = dict()
        if port not in switches[switch]:
            switches[switch][port] = slope
        else:
            switches[switch][port] += slope
    for switch in switches:
        for port in switches[switch]:
            if switches[switch][port] > 750000000:
                print("WARN: Overreservation on switch " + switch + " port " + str(port) + " with idle slope " + str(switches[switch][port]) + " Mbit/s")

def countServices(services):
    numServices = len(services)
    print("Number of services: " + str(numServices))
    numSubs = {
        "total": 0,
        "min": 1000000,
        "max": 0,
        "avg": 0
    }
    for service in services:
        dests = len(service['dstHosts'])
        if dests < numSubs["min"]:
            numSubs["min"] = dests
        if dests > numSubs["max"]:
            numSubs["max"] = dests
        numSubs["total"] += dests
    numSubs["avg"] = numSubs["total"] / numServices
    print("Number of subscribers: " + str(numSubs))


In [4]:
# read json files 
files = {
    "flowUpdatesFixedCMI.json",
    "flowUpdatesStreamCMI.json",
    "flowUpdatesFixedCMIB500.json",
    "flowUpdatesFixedCMIB1000.json"
}
for file in files:
    print("Analysing " + file)
    data = json.load(open(file))
    warnOverReservation(data["finalConfig"]["switchIdleSlopes"])
    countServices(data["finalConfig"]["services"])
    print()

Analysing flowUpdatesFixedCMI.json
WARN: Overreservation on switch 0A-AA-00-00-00-16 port 1 with idle slope 885504000 Mbit/s
WARN: Overreservation on switch 0A-AA-00-00-00-1B port 2 with idle slope 1076064000 Mbit/s
Number of services: 212
Number of subscribers: {'total': 448, 'min': 1, 'max': 4, 'avg': 2.1132075471698113}

Analysing flowUpdatesStreamCMI.json
WARN: Overreservation on switch 0A-AA-00-00-00-16 port 1 with idle slope 785476924 Mbit/s
Number of services: 212
Number of subscribers: {'total': 448, 'min': 1, 'max': 4, 'avg': 2.1132075471698113}

Analysing flowUpdatesFixedCMIB500.json
WARN: Overreservation on switch 0A-AA-00-00-00-16 port 1 with idle slope 885504000 Mbit/s
WARN: Overreservation on switch 0A-AA-00-00-00-1B port 2 with idle slope 916464000 Mbit/s
Number of services: 212
Number of subscribers: {'total': 448, 'min': 1, 'max': 4, 'avg': 2.1132075471698113}

Analysing flowUpdatesFixedCMIB1000.json
WARN: Overreservation on switch 0A-AA-00-00-00-16 port 1 with idle sl